# Modules

In [1]:
import json
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import re
import requests

from bs4 import BeautifulSoup as bs

# Constantes

In [2]:
PATH_DATA = r"C:\Users\ahoun\Documents\Documents\Nuku Data\ETL-real-estate-prices\Data"
PATH_DATA_RAW = rf"{PATH_DATA}\Citya\Raw"

# Application

In [3]:
if os.path.exists(PATH_DATA):
    print('Le dossier Data existe déja.')
else: 
    os.makedirs(PATH_DATA)
    print('Le dossier Data a été créé avec succès.')

Le dossier Data existe déja.


In [4]:
if os.path.exists(PATH_DATA_RAW):
    print('Le dossier Data Raw existe déja.')
else: 
    os.makedirs(PATH_DATA_RAW)
    print('Le dossier Data Raw a été créé avec succès.')

Le dossier Data Raw existe déja.


In [5]:
if os.path.exists(PATH_DATA_RAW + "/Citya/json"):
    print('Le dossier Data Raw citya existe déja.')
else: 
    os.makedirs(PATH_DATA_RAW + "/Citya/json")
    print('Le dossier Data Raw citya a été créé avec succès.')
    
if os.path.exists(PATH_DATA_RAW + "/Citya/parquet"):
    print('Le dossier Data Raw citya existe déja.')
else: 
    os.makedirs(PATH_DATA_RAW + "/Citya/parquet")
    print('Le dossier Data Raw citya a été créé avec succès.')

Le dossier Data Raw citya existe déja.
Le dossier Data Raw citya existe déja.


## Téléchargement données d'une page

In [6]:
page = 1
url = fr"https://www.citya.com/annonces/vente/appartement,maison?sort=b.dateMandat&direction=desc&page={page}"
response = requests.get(url)

html = response.content
soup = bs(html, "html")
# print("Status code: ", response.status_code)
descs = soup.find_all('div', {'class': "infos"})
# print("DESCS: ", descs)

data_citya = []
    

for i,des in enumerate(descs):
    content = {'id': i,
                'title': None,
                'description': None,
                'type': None,
                'surface': None,
                'price': None,
                'city': None,
                'postal_code': None,
                'number_pieces': None,
                }
    
    # print("DES: ", des)
    content['title'] = des.find('h3').get_text().strip().replace('\n','')
    
    
    content['description'] = des.find('p', class_="description-start").get_text().strip()
    content['description'] = content['description'].replace('\xa0','').replace('[...]', '')
    # print("DESCRIPTION: ", content['description'])
    
    cities = des.find('p', class_="ville").get_text().strip()
    content['city'] = " ".join(re.findall('[a-zA-ZÀ-ȕ]+', cities))
    # print("CITY: ", content['city'])
    
    content['postal_code'] = re.findall('[0-9]+', cities)[-1]
    # print("POSTAL CODE: ", content['postal_code'])
    
    content['price'] = int(des.find('p', class_="prix").find('strong').get_text().replace('\u202f','').replace('\xa0€','').split()[0])
    # print("PRICE: ", content['price'])
    
    content['type'] = des.find('h3').get_text().split()[0]
    # print("TYPE: ", content['type'])
    
    content['number_pieces'] = int(des.find('h3').find('strong').get_text().split()[0])
    # print("NUMBER PIECES: ", content['number_pieces'])
    
    content['surface'] = des.find('h3').get_text().split()[-1]
    # print("SURFACE: ", content['surface'])


    data_citya.append(content)

In [7]:
data_citya

[{'id': 0,
  'title': 'Appartement                     3 pièces 65m²',
  'description': "MOULIN JOLI LA POSSESSION  APPARTEMENT T 3  RESIDENCE SECURISEE - Citya-La Possession .Secteur Moulin Joli .Dans  résidence \n                            sécurisée , Bel appartement de type T3  ,de  65 m²  + varangue de 7m²  ,bonnes  prestations. 2 chambres climatisées avec rangement ,séjour ,cuisine ,salle de bain.1 place de parking sécurisée, privative. Secteur proche des écoles ,des commerces. Pour investissement ,appartement loué.  .Copropriété de 30 Lots ,charges 385€/trimestre .Belle  opportunité. A vendre 173 000€ .Contact pour visite: Laurent Sanchez 06 92 85 04 50  ou lasanchez@citya.com.  Le bien est soumis au statut de la copropriété , la copropriété comporte 30 lots , la quote-part budget prévisionnel (dépenses courantes) est de 1 460,00 euros/an. \n Honoraires à la charge du vendeur. \nVous pouvez consulter les barèmes d'honoraires à l'adresse suivante : https://www.citya.com/agences-i

## Téléchargement données de plusieurs pages

In [8]:
# page = 1
# url = fr"https://www.citya.com/annonces/vente/appartement,maison?sort=b.dateMandat&direction=desc&page={page}"
# response = requests.get(url)

# while response.status_code == 200 :
#     html = response.content
#     soup = bs(html, "html")
#     # print("Status code: ", response.status_code)
#     descs = soup.find_all('div', {'class': "infos"})
#     # print("DESCS: ", descs)

#     data_citya = []
        

#     for i,des in enumerate(descs):
#         content = {'id': i,
#                     'title': None,
#                     'description': None,
#                     'type': None,
#                     'surface': None,
#                     'price': None,
#                     'city': None,
#                     'postal_code': None,
#                     'number_pieces': None,
#                     }
        
#         # print("DES: ", des)
#         content['title'] = des.find('h3').get_text().strip().replace('\n','')
        
        
#         content['description'] = des.find('p', class_="description-start").get_text() + "\n" + \
#                                 des.find('span', class_="description-more").get_text()
#         content['description'] = content['description'].replace('\xa0','')
#         # print("DESCRIPTION: ", content['description'])
        
        
#         cities = des.find('p', class_="ville").get_text().strip()
        
#         content['city'] = " ".join(re.findall('[a-zA-ZÀ-ȕ]+', cities))
#         # print("CITY: ", content['city'])
        
#         content['postal_code'] = re.findall('[0-9]+', cities)[-1]
#         # print("POSTAL CODE: ", content['postal_code'])
        
#         content['price'] = int(des.find('p', class_="prix").find('strong').get_text().replace('\u202f','').replace('\xa0€','').split()[0])
#         # print("PRICE: ", content['price'])
        
#         content['type'] = des.find('h3').get_text().split()[0]
#         # print("TYPE: ", content['type'])
        
#         content['number_pieces'] = int(des.find('h3').find('strong').get_text().split()[0])
#         # print("NUMBER PIECES: ", content['number_pieces'])
        
#         content['surface'] = des.find('h3').get_text().split()[-1]
#         # print("SURFACE: ", content['surface'])


#         data_citya.append(content)
        
        
#         page_path = rf"{PATH_DATA_RAW}/Citya/"

#         nom_fichier = rf"{page_path}/json/data_page_{page}.json"
#         if os.path.exists(nom_fichier):
#             print(f'Le fichier data_page_{page} existe déja.\n')
#         else: 
#             with open(nom_fichier, "w", encoding="utf-8") as fichier:  
#                 json.dump(data_citya, fichier, ensure_ascii=False)
#                 fichier.close()
                
        
#         parquet_path = rf"{page_path}/parquet/data_page_{page}.parquet"
#         if os.path.exists(parquet_path):
#             print(f'Le fichier data_page_{page}.parquet existe déjà.\n')
#         else:
#             # Convertir la liste en un DataFrame pandas
#             df = pd.DataFrame(data_citya)

#             # Enregistrer le DataFrame au format Parquet
#             table = pa.Table.from_pandas(df)
#             pq.write_table(table, parquet_path, compression='gzip')
#             print(f'Les données ont été enregistrées au format Parquet dans {parquet_path}.\n')

        
        
#         page += 1